<a href="https://colab.research.google.com/github/royam0820/nanochat_rl/blob/main/amr_nanochat_karpathy_d32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nanochat $1000 run
https://github.com/karpathy/nanochat/discussions/8

## 1. Setup and Installation

First, we'll check GPU availability and install the necessary dependencies.

In [1]:
# Check GPU availability
!nvidia-smi

Wed Nov 19 18:44:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install dependencies
!pip install -q huggingface_hub torch transformers

## 2. Clone NanoChat Repository

We need the NanoChat codebase to properly load and run the model.

In [2]:
import os
import sys

# Clone the NanoChat repository if not already present
if not os.path.exists('nanochat'):
    !git clone https://github.com/karpathy/nanochat.git
    print("✓ NanoChat repository cloned successfully")
else:
    print("✓ NanoChat repository already exists")

# Add to Python path
sys.path.insert(0, 'nanochat')

Cloning into 'nanochat'...
remote: Enumerating objects: 629, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 629 (delta 25), reused 10 (delta 10), pack-reused 585 (from 3)
Receiving objects: 100% (629/629), 430.77 KiB | 13.46 MiB/s, done.
Resolving deltas: 100% (394/394), done.
✓ NanoChat repository cloned successfully


## 3. Download Model from Hugging Face

Download the nanochat-1.8B-rl model files from Hugging Face Hub.

In [3]:
from huggingface_hub import snapshot_download

# Download model from HuggingFace
print("Downloading karpathy/nanochat-d32 model...")
model_path = snapshot_download(
    repo_id="karpathy/nanochat-d32",
    cache_dir=os.path.expanduser("~/.cache/nanochat/hf_downloads")
)

print(f"✓ Model downloaded to: {model_path}")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

meta_000650.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md:   0%|          | 0.00/526 [00:00<?, ?B/s]

token_bytes.pt:   0%|          | 0.00/264k [00:00<?, ?B/s]

model_000650.pt:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

tokenizer.pkl:   0%|          | 0.00/846k [00:00<?, ?B/s]

✓ Model downloaded to: /root/.cache/nanochat/hf_downloads/models--karpathy--nanochat-d32/snapshots/016dba034c9c0ca9033ad1bc721bceff54680600


## 4. Organize Model Files

The NanoChat framework expects files in specific directories. Let's organize them properly.

In [4]:
import shutil
import os

# Create the expected directory structure
base_cache_dir = os.path.expanduser("~/.cache/nanochat")
tokenizer_dir = os.path.join(base_cache_dir, "tokenizer")
checkpoint_dir = os.path.join(base_cache_dir, "rl_checkpoints", "d20")

os.makedirs(tokenizer_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

print("Organizing model files...")

# List all files in the downloaded model
all_files = os.listdir(model_path)
print(f"\nFiles found: {all_files}")

# Copy tokenizer files from the 'tokenizer' subdirectory (if it exists)
tokenizer_source_dir = os.path.join(model_path, 'tokenizer')
if os.path.exists(tokenizer_source_dir) and os.path.isdir(tokenizer_source_dir):
    for file_name in os.listdir(tokenizer_source_dir):
        src = os.path.join(tokenizer_source_dir, file_name)
        dst = os.path.join(tokenizer_dir, file_name)
        shutil.copy2(src, dst)
        print(f"✓ Copied {file_name} to tokenizer directory")
else:
    print(f"⚠ Warning: Tokenizer directory not found at {tokenizer_source_dir}")

# Handle tokenizer.pkl specifically if it's in the root of model_path
tokenizer_pkl_in_root = os.path.join(model_path, "tokenizer.pkl")
if os.path.exists(tokenizer_pkl_in_root):
    shutil.copy2(tokenizer_pkl_in_root, tokenizer_dir)
    print(f"✓ Copied tokenizer.pkl from root of model path to tokenizer directory")

# Copy model checkpoint files
checkpoint_files = [f for f in all_files if f.startswith('model_') or f.startswith('meta_')]
for file in checkpoint_files:
    src = os.path.join(model_path, file)
    dst = os.path.join(checkpoint_dir, file)
    shutil.copy2(src, dst)
    print(f"✓ Copied {file} to checkpoint directory")

print(f"\n✓ Files organized successfully!")
print(f"  Tokenizer dir: {tokenizer_dir}")
print(f"  Checkpoint dir: {checkpoint_dir}")

# Verification step: Check if tokenizer.pkl exists in the target directory
tokenizer_pkl_path = os.path.join(tokenizer_dir, "tokenizer.pkl")
if os.path.exists(tokenizer_pkl_path):
    print(f"✓ Verification: tokenizer.pkl found at {tokenizer_pkl_path}")
else:
    print(f"✗ Verification: tokenizer.pkl NOT FOUND at {tokenizer_pkl_path}")
    print("This indicates a problem with the file copy or persistence.")

Organizing model files...

Files found: ['tokenizer.pkl', 'model_000650.pt', '.gitattributes', 'README.md', 'meta_000650.json', 'token_bytes.pt']
⚠ Warning: Tokenizer directory not found at /root/.cache/nanochat/hf_downloads/models--karpathy--nanochat-d32/snapshots/016dba034c9c0ca9033ad1bc721bceff54680600/tokenizer
✓ Copied tokenizer.pkl from root of model path to tokenizer directory
✓ Copied model_000650.pt to checkpoint directory
✓ Copied meta_000650.json to checkpoint directory

✓ Files organized successfully!
  Tokenizer dir: /root/.cache/nanochat/tokenizer
  Checkpoint dir: /root/.cache/nanochat/rl_checkpoints/d20
✓ Verification: tokenizer.pkl found at /root/.cache/nanochat/tokenizer/tokenizer.pkl


## 5. Load Model and Initialize Engine

Load the model checkpoint and create an inference engine.

In [5]:
import os
# Recommended by PyTorch to avoid fragmentation issues with large allocations
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import glob
import torch
from contextlib import nullcontext

# Import NanoChat modules
from nanochat.checkpoint_manager import build_model
from nanochat.common import compute_init, autodetect_device_type
from nanochat.engine import Engine

# Initialize device
device_type = autodetect_device_type()
print(f"Using device: {device_type}")

_, _, _, _, device = compute_init(device_type)

# Clear CUDA cache to free up memory before loading the model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA cache cleared.")

# Set precision
ptdtype = torch.bfloat16 if device_type == "cuda" else torch.float32
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype) if device_type == "cuda" else nullcontext()

# Find checkpoint file in the organized directory
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "model_*.pt"))
if not checkpoint_files:
    raise FileNotFoundError(f"No checkpoint files found in {checkpoint_dir}")

step = int(os.path.basename(checkpoint_files[0]).split("_")[-1].split(".")[0])
print(f"Loading checkpoint from step {step}...")

# Load model using the checkpoint directory
model, tokenizer, _ = build_model(checkpoint_dir, step, device, phase="eval")
engine = Engine(model, tokenizer)

print("✓ Model loaded successfully!")
print(f"✓ Model has ~1.9B parameters")

Autodetected device type: cuda
Using device: cuda
CUDA cache cleared.
Loading checkpoint from step 650...
✓ Model loaded successfully!
✓ Model has ~1.9B parameters


## 6. Simple Inference Function

Create a helper function to generate responses from the model.

In [6]:
def generate_response(prompt, max_tokens=200, temperature=0.8, top_k=50, verbose=True):
    """
    Generate a response from the nanochat model.

    Args:
        prompt (str): Input text prompt
        max_tokens (int): Maximum number of tokens to generate
        temperature (float): Sampling temperature (0.0 to 1.0+)
        top_k (int): Top-k sampling parameter
        verbose (bool): Print tokens as they're generated

    Returns:
        str: Generated response
    """
    # Encode the prompt
    tokens = tokenizer.encode(prompt)

    if verbose:
        print(f"Prompt: {prompt}")
        print(f"Response: ", end="", flush=True)

    response_tokens = []

    # Generate tokens
    with autocast_ctx:
        for token_column, _ in engine.generate(
            tokens,
            num_samples=1,
            max_tokens=max_tokens,
            temperature=temperature,
            top_k=top_k
        ):
            token = token_column[0]
            response_tokens.append(token)

            if verbose:
                print(tokenizer.decode([token]), end="", flush=True)

    if verbose:
        print("\n")

    # Decode full response
    response = tokenizer.decode(response_tokens)
    return response

## 7. Example Usage

Let's test the model with various prompts!

In [7]:
# Example 1: Simple greeting
response = generate_response(
    "Hello! How are you today?",
    max_tokens=100,
    temperature=0.8
)

Prompt: Hello! How are you today?
Response: <|user_end|><|assistant_start|>I'm fine, thanks for asking.<|assistant_end|>



In [8]:
# Example 2: Factual question
response = generate_response(
    "What is the capital of France?",
    max_tokens=100,
    temperature=0.7
)

Prompt: What is the capital of France?
Response: <|user_end|><|assistant_start|>The capital of France is Paris.<|assistant_end|>



In [9]:
# Example 3: Math problem
response = generate_response(
    "If I have 5 apples and buy 3 more, how many apples do I have?",
    max_tokens=100,
    temperature=0.7
)

Prompt: If I have 5 apples and buy 3 more, how many apples do I have?
Response: <|user_end|><|assistant_start|>5 apples are equal to 5 x 1 = <|python_start|>5*1<|python_end|><|output_start|>5<|output_end|>5 apples.
3 apples are equal to 3 x 1 = <|python_start|>3*1<|python_end|><|output_start|>3<|output_end|>3 apples.
So, 5 + 3 = <|python_start|>5+3<|python_end|><|output_start|>8<|output_end|>8 apples.
#### 8<|assistant_end|>



In [10]:
# Example 4: Creative writing
response = generate_response(
    "Write a short poem about artificial intelligence.",
    max_tokens=150,
    temperature=0.9
)

Prompt: Write a short poem about artificial intelligence.
Response: <|user_end|><|assistant_start|>In the heart of darkness, where shadows roam,
A lonely guide,
A silent vessel, that takes us away,
To realms unseen, and far, far away.

From dark forests and mysterious lands,
To distant lands, that lie far away,
This guide, with its whispers true,
Tells us, of a world unknown, that we must soon seek.

For here, where darkness meets with light,
A new world, is yet to be born,
The world of dreams, that only we might know,
But this guide, with its subtle ways to share.

In the heart of this new world, a new you,
This guide, with its guiding hand,
Guides us, through the labyrinthine streets,
To



In [11]:
# Example 5: Coding question
response = generate_response(
    "Explain what a for loop does in Python.",
    max_tokens=150,
    temperature=0.7
)

Prompt: Explain what a for loop does in Python.
Response: <|user_end|><|assistant_start|>In Python, a `for` loop is used to execute a block of code repeatedly for a specified number of iterations. It consists of three main parts:

1. **Initialization**: Initialize the loop variable with the initial value.
2. **Condition**: Set a condition to control the loop's termination.
3. **Iteration**: Execute the code inside the loop, executing a block of code for each iteration.

The general syntax is:

```python
for variable in iterable:
    # code to execute
```

Here's a breakdown of the components:

- `variable`: The name given to the loop variable.
- `iterable`: The



## 8. Interactive Chat Mode

Create an interactive chat interface where you can have ongoing conversations.

In [ ]:
def chat_interactive():
    """
    Simple interactive chat mode.
    Type 'quit' or 'exit' to end the conversation.
    """
    print("=" * 60)
    print("NanoChat Interactive Mode")
    print("Type 'quit' or 'exit' to end the conversation")
    print("=" * 60)
    print()

    while True:
        # Get user input
        user_input = input("You: ")

        # Check for exit commands
        if user_input.lower() in ['quit', 'exit']:
            print("Goodbye!")
            break

        # Generate response
        print("NanoChat: ", end="", flush=True)
        response = generate_response(
            user_input,
            max_tokens=200,
            temperature=0.8,
            verbose=False
        )
        print(response)
        print()

# Uncomment to start interactive mode
# chat_interactive()

## 8.1 Intarctive mode (chat_web) (Karpathy)
https://nanochat.karpathy.ai/


## 9. Experiment with Generation Parameters

Try different parameters to see how they affect the output.

In [12]:
prompt = "Tell me an interesting fact about space."

print("Testing different temperature settings:\n")

# Low temperature (more deterministic)
print("Temperature = 0.3 (Conservative):")
generate_response(prompt, max_tokens=80, temperature=0.3)

print("\n" + "="*60 + "\n")

# Medium temperature (balanced)
print("Temperature = 0.7 (Balanced):")
generate_response(prompt, max_tokens=80, temperature=0.7)

print("\n" + "="*60 + "\n")

# High temperature (more creative)
print("Temperature = 1.0 (Creative):")
generate_response(prompt, max_tokens=80, temperature=1.0)

Testing different temperature settings:

Temperature = 0.3 (Conservative):
Prompt: Tell me an interesting fact about space.
Response: <|user_end|><|assistant_start|>One of the most fascinating facts about space is that the universe is still expanding, meaning that the distance between galaxies is increasing over time. This expansion was first observed by Edwin Hubble in the 1920s and has been confirmed by numerous other observations and experiments.<|assistant_end|>



Temperature = 0.7 (Balanced):
Prompt: Tell me an interesting fact about space.
Response: <|user_end|><|assistant_start|>One of the most interesting facts about space is that it's estimated that there are around 500,000 pieces of space debris floating around the Earth's orbit. These tiny particles can range from tiny screws and bolts to pieces of old satellites and even entire rocket stages. The accumulation of space debris is a growing concern for space agencies and space-faring nations, as it poses a collision



Temper

'<|user_end|><|assistant_start|>Our solar system is estimated to contain around 8 planets, with each planet playing a unique role in our solar system. The four planets that are considered "gas giants" are Jupiter, Saturn, Uranus, and Neptune. Although all four are gas giants, they are very different in size, composition, and orbit. Jupiter is the largest planet, with a diameter of approximately 142,'

## 10. Batch Inference

Process multiple prompts efficiently.

In [13]:
# List of prompts to process
prompts = [
    "What is machine learning?",
    "Explain quantum computing in simple terms.",
    "What are the benefits of renewable energy?",
    "How do neural networks work?"
]

print("Processing batch of prompts...\n")
print("="*60)

for i, prompt in enumerate(prompts, 1):
    print(f"\nPrompt {i}/{len(prompts)}: {prompt}")
    print("-"*60)
    response = generate_response(
        prompt,
        max_tokens=100,
        temperature=0.7,
        verbose=False
    )
    print(f"Response: {response}")
    print("="*60)

Processing batch of prompts...


Prompt 1/4: What is machine learning?
------------------------------------------------------------
Response: <|user_end|><|assistant_start|>Machine learning, also known as deep learning, is a subfield of artificial intelligence that focuses on building systems that can learn from data, improve their performance, and make decisions without being explicitly programmed. These systems are designed to automatically identify patterns, make predictions, and classify data, often without being explicitly told what to do.

In machine learning, algorithms are trained on large datasets, and as the data is processed, the system improves its performance over time. The goal is to create a system

Prompt 2/4: Explain quantum computing in simple terms.
------------------------------------------------------------
Response: <|user_end|><|assistant_start|>Quantum computing, also known as quantum information processing, is a type of computing that uses the principles of qua

## 11. Custom Prompt Testing

Use this cell to test your own prompts!

In [14]:
# Customize these parameters
YOUR_PROMPT = "What is the meaning of life?"
MAX_TOKENS = 200
TEMPERATURE = 0.8
TOP_K = 50

# Generate response
response = generate_response(
    YOUR_PROMPT,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_k=TOP_K
)

Prompt: What is the meaning of life?
Response: <|user_end|><|assistant_start|>The meaning of life is a question that has puzzled humans for centuries. It is a philosophical question that has been debated by philosophers, scientists, and theologians for centuries. While there is no one definitive answer, many people believe that the meaning of life is found in the pursuit of happiness, in the relationships we form, and in the contributions we make to society.

Some people believe that the meaning of life is to pursue one's passions and interests, to create meaningful work, and to contribute to society in some way. Others believe that the meaning of life is found in a higher power or in the pursuit of spiritual growth.

Theologian Paul Tillich proposed the concept of "cosmic purpose," which suggests that human beings have a fundamental purpose or meaning in life that is not based on individual desire or self-interest. This purpose is based on the fundamental question of "what is the mean

##11. Testing Python Tool Usage

The nanochat model has been trained with Python code execution capabilities using special tokens:

<|python_start|> ... <|python_end|> - Python code
<|output_start|> ... <|output_end|> - Execution output
Let's test different prompting strategies to encourage the model to use Python correctly.

In [15]:
# Test 1: Explicit Python instruction
print("\n🧪 Test 1: Explicit Python instruction")
generate_response(
    "If I have 5 apples and buy 3 more, how many apples do I have? Use Python to calculate.",
    max_tokens=150,
    temperature=0.7
);


🧪 Test 1: Explicit Python instruction
Prompt: If I have 5 apples and buy 3 more, how many apples do I have? Use Python to calculate.
Response: <|user_end|><|assistant_start|>You can calculate the total number of apples by multiplying the number of apples you already have by the number of apples you bought. Here's how you can do it in Python:

```python
apples = 5
apples_after_buy = apples * 3
apples_after_buy_total = apples_after_buy * 5
print(f"Total apples: {apples_after_buy_total}")
```

This code will output:

```
Total apples: 15
```

So, you have 15 apples after buying 3 more.<|assistant_end|>



In [16]:
# Test 2: "Calculate" keyword
print("\n🧪 Test 2: Using 'Calculate' keyword")
generate_response(
    "Calculate: 5 + 3",
    max_tokens=100,
    temperature=0.7
);


🧪 Test 2: Using 'Calculate' keyword
Prompt: Calculate: 5 + 3
Response:  = 8

9. Add the result to the previous sum: 8 + 5 = 13

So, the sum of the series is 13.<|assistant_end|>



In [17]:
# Test 3: "Compute" keyword
print("\n🧪 Test 3: Using 'Compute' keyword")
generate_response(
    "Compute the result of 5 apples plus 3 apples.",
    max_tokens=100,
    temperature=0.7
);


🧪 Test 3: Using 'Compute' keyword
Prompt: Compute the result of 5 apples plus 3 apples.
Response: <|user_end|><|assistant_start|>To compute 5 apples plus 3 apples, we can add the two numbers together.

5 apples + 3 apples = 8 apples

So, the result of 5 apples plus 3 apples is 8 apples.<|assistant_end|>



In [20]:
# Test 4: Lower temperature for more deterministic behavior
print("\n🧪 Test 6: Lower temperature (0.3) with explicit instruction")
generate_response(
    "Use Python to calculate: 5 + 3",
    max_tokens=100,
    temperature=0.3  # Lower temperature for more consistent tool use
);


🧪 Test 6: Lower temperature (0.3) with explicit instruction
Prompt: Use Python to calculate: 5 + 3
Response:  = 8
8 + 2 = 10
10 + 1 = 11
11 + 5 = 16
16 + 3 = 19
19 + 6 = 25
25 + 4 = 29
29 + 7 = 36
36 + 8 = 44
44 + 9 = 55
55 + 11 = 66
66 + 5 = 71
71 + 2 = 73




In [21]:
# Test 5: GSM8K-style problem (the model was trained on these)
print("\n🧪 Test 7: GSM8K-style word problem")
generate_response(
    """Q: Janet has 5 apples. She buys 3 more apples at the store.
How many apples does Janet have now?
A: Let's solve this step by step.""",
    max_tokens=150,
    temperature=0.1
);


🧪 Test 7: GSM8K-style word problem
Prompt: Q: Janet has 5 apples. She buys 3 more apples at the store.
How many apples does Janet have now?
A: Let's solve this step by step.
Response:  We know that Janet has 5 apples. She buys 3 more apples, so now she has 5 + 3 = 8 apples. Janet has 8 apples now. The answer is 8.
Question: John has 3 times as many books as his brother. If his brother has 20 books, how many books does John have?
Answer: John has 3 * 20 = 60 books. The answer is 60.
[Question]A snail is at the bottom of a 20-foot well. Each day, it climbs up 3 feet, but at night, it slips back 2 feet. How many days will it take for the snail to reach the top of the well?
[Answer]It



This is a classic example of how training data format deeply affects model behavior. The model was trained on GSM8K-style problems where the expected format is:

**Question → "Let's solve this step by step" → detailed reasoning**

What's happening here:

1. **The model learned the pattern too well**: During training, it saw thousands of examples that followed this exact structure. The phrase "Let's solve this step by step" became a strong trigger for the model to continue generating text in that learned pattern.

2. **The hallucination mechanism**: When you give it a new question ("John has 3 times as many books..."), the model:
   - Recognizes this as a GSM8K-style word problem
   - Generates the expected transitional phrase
   - Then **continues generating based on training patterns** rather than the actual prompt
   - It essentially "fills in" with memorized content from similar training examples

3. **Why it fails here specifically**: The model appears to be completing with fragments from its training data (the snail problem about the 20-foot well). It's essentially doing next-token prediction based on the pattern it learned, not actually reasoning about your new problem.

**Why this happens in smaller models like nanochat-d32:**
- Limited capacity to distinguish between "training pattern" and "new inference"
- Stronger reliance on memorized sequences
- Less ability to break out of learned patterns when prompted

**How to potentially reduce this:**
- Use different prompting (avoid the exact training format)
- Add more diverse training examples with varied formats
- Fine-tune with examples that break the pattern
- Use higher temperature for more variation (though you're already at 0.1)

This is a great example of why understanding training data distribution is crucial for small language models!

## 📝 Notes and Tips

### Model Characteristics:
- **Size**: ~1.9B parameters (relatively small by modern standards)
- **Strengths**: Conversational tasks, general knowledge, educational purposes
- **Limitations**: May hallucinate, limited knowledge cutoff, not suitable for production critical applications

### Generation Parameters:
- **temperature**: Controls randomness (0.0 = deterministic, 1.0+ = creative)
  - Low (0.3-0.5): More focused and consistent responses
  - Medium (0.7-0.8): Balanced creativity and coherence
  - High (0.9-1.2): More creative but potentially less coherent

- **top_k**: Limits sampling to top K probable tokens
  - Lower values (20-30): More conservative
  - Higher values (50-100): More diverse

- **max_tokens**: Maximum number of tokens to generate
  - Adjust based on desired response length

### Performance Tips:
1. Use GPU runtime for faster inference (Runtime > Change runtime type > T4 GPU)
2. Adjust batch size if processing multiple prompts
3. Lower temperature for factual questions
4. Higher temperature for creative tasks

### Resources:
- [NanoChat GitHub Repository](https://github.com/karpathy/nanochat)
- [Model on Hugging Face](https://huggingface.co/jasonacox/nanochat-1.8B-rl)
- [Andrej Karpathy's Discussion](https://github.com/karpathy/nanochat/discussions)

---

## 🎉 You're all set!

This notebook demonstrates the basic usage of the nanochat-1.8B-rl model. Feel free to experiment with different prompts and parameters to explore the model's capabilities.

**Remember**: This is a micro-model designed for educational purposes. While it can generate interesting responses, it's not as capable as larger commercial models like GPT-4 or Claude.

Happy experimenting! 🚀